In [1]:
from pulp import *
import numpy as np

## Sudoku rules

The classic Sudoku game involves a grid of 81 squares. The grid is divided into nine blocks, each containing nine squares

The rules of the game are simple: 
* Each of the nine blocks has to contain all the numbers 1-9 within its squares. 
* Each number can only appear once in a row, column or box.

source: http://www.counton.org/sudoku/rules-of-sudoku.php

In [2]:
problem_board =    [[0, 0, 0, 6, 0, 0, 0, 1, 0],
                   [0, 7, 2, 0, 0, 0, 0, 0, 0],
                   [0, 8, 0, 0, 4, 0, 0, 0, 0],
                   [5, 0, 0, 3, 0, 9, 0, 0, 0],
                   [0, 0, 0, 0, 7, 0, 8, 0, 0],
                   [6, 0, 0, 0, 0, 0, 0, 0, 0],
                   [3, 0, 0, 9, 0, 0, 0, 0, 5],
                   [0, 4, 0, 0, 0, 0, 7, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0]]

## Mathematical programming formulation
by Francesco Bucci &copy;

### Notation and parameters
* $s$: size of one of the board dimentions
* $R = \{1,2, ..., s\}$: is the set of rows
* $C = \{1,2, ..., s\}$: is the set of columns
* $N = \{1,2, ..., s\}$: is the set of allowed numbers
* $B$: is the set of tuples $(row, column, number)$ with the given numbers on the initial board  
* $L = \{ \{1,2,3\}, \{4,5,6\}, ... , \{s-3,s-1,s\}\}$: are set of set to help define the sub-blocks of the board
* $B_{(l_i,l_j)} = \{(i,j) | i \in l_i,\; j \in l_j\}, l_i \in L,\; l_j \in L$ : contains the $(row, column)$ tuples for each blocks of the board



In [14]:
s = 9 # size of one of the board dimentions
C = set(range(1,s+1))
R = set(range(1,s+1))
N = set(range(1,s+1))
B={}

L=list()
for l in range(1,s,3):
    L.append(set(range(l,l+3)))
    
Bl = {}
count =1
for li in L:
    for lj in L:
        Bl[count] = set([(i,j) for i in li for j in lj])
        count +=1
        
    
# Create tuples from initial board
for i in range(s):
    for j in range(s):
        if problem_board[i][j]>0:
            B[(i+1,j+1)] = problem_board[i][j]


### Decision variables

* $b_{i,j}^n,\in \{0,1\},  i\in R,  j\in C, n\in N$: takes value 1 if in cell $i,j$ number $n$ is assigned, otherwise 0
*

In [4]:
prob = LpProblem()
b_ij_n = {}
for n in N:
    for i in R:
        for j in C:
            b_ij_n[(i,j,n)] = LpVariable("b_%s,%s_%s"%(i,j,n),lowBound=0,upBound=1,cat="Binary ")

## Constraints

$$ \sum_{j \in C} b_{ij}^n = 1,\forall\:n \in N,\forall\:i \in R\;\;(1.)$$ Each number can appear only once in each row
$$ \sum_{i \in R} b_{ij}^n = 1,\forall\:n \in N,\forall\:j \in C\;\;(2.)$$ Each number can appear only once in each column
$$ \sum_{ (i,j) \in l} b_{ij}^n = 1,\forall\:n \in N,\forall\:l \in Bl\;\;(3.)$$ Each number can appear only once in each 3x3 square
$$ \sum_{n \in N} b_{ij}^n = 1,\forall\:i \in R,\forall\:j \in C\;\;(4.)$$ Each cell can only be activated for one of the n boards
$$  b_{ij}^n = 1,\forall\:(i,j,n) \in B\;(5.)$$ Assign variable values from the initial board

In [5]:
#1 each number can appear only once in each row
for n in N: # for each number
    for i in R: # for each row
        prob += lpSum([b_ij_n[(i,j,n)] for j in C]) == 1
        
#2 each number can appear only once in each column
for n in N: # for each number
    for j in C: # for each column
        prob += lpSum([b_ij_n[(i,j,n)] for i in R]) == 1
        
#3 each number can appear only once in each 3x3 square
for n in N:
    for l in Bl:
        prob += lpSum(b_ij_n[(i,j,n)] for (i,j) in Bl[l]) == 1
        
#4 each cell can only be activated for one of the n boards:
# it means that there can be only one number in a given cell
for i in R:
    for j in C:
        prob += lpSum(b_ij_n[(i,j,n)] for n in N) == 1
        
#5 Assign variable values from the initial board
for (i,j) in B:
    prob += b_ij_n[(i,j,B[(i,j)])] == 1

In [6]:
# might be redundant as an objective function
# It will minimise the number of cells filled with numbers
# which is going to be always 81 if the problem is solved correcty
prob += lpSum([b_ij_n[(i,j,n)] for i in R for j in C for n in N])

In [7]:
solver = PULP_CBC_CMD(maxSeconds=100)

In [8]:
prob.solve(solver=solver)

1

In [9]:
# Extract solution
final_board = []
for i in R:
    for j in C:
        final_board.append([b_ij_n[(i,j,n)].name[6] for n in N if b_ij_n[(i,j,n)].varValue==1])

Print and check the solution

In [10]:
solution = np.array(final_board).reshape((9,9))

In [11]:
print(solution)

[['4' '5' '3' '6' '2' '8' '9' '1' '7']
 ['1' '7' '2' '5' '9' '3' '6' '4' '8']
 ['9' '8' '6' '7' '4' '1' '5' '2' '3']
 ['5' '1' '7' '3' '8' '9' '2' '6' '4']
 ['2' '3' '4' '1' '7' '6' '8' '5' '9']
 ['6' '9' '8' '4' '5' '2' '3' '7' '1']
 ['3' '2' '1' '9' '6' '7' '4' '8' '5']
 ['8' '4' '9' '2' '1' '5' '7' '3' '6']
 ['7' '6' '5' '8' '3' '4' '1' '9' '2']]


In [12]:
b = np.array( [['4', '5', '3', '6', '2', '8', '9', '1', '7'],
               ['1', '7', '2', '5', '9', '3', '6', '4', '8'],
               ['9', '8', '6', '7', '4', '1', '5', '2', '3'],
               ['5', '1', '7', '3', '8', '9', '2', '6', '4'],
               ['2', '3', '4', '1', '7', '6', '8', '5', '9'],
               ['6', '9', '8', '4', '5', '2', '3', '7', '1'],
               ['3', '2', '1', '9', '6', '7', '4', '8', '5'],
               ['8', '4', '9', '2', '1', '5', '7', '3', '6'],
               ['7', '6', '5', '8', '3', '4', '1', '9', '2']], dtype='<U1')

In [13]:
sum(sum(solution == b)) == 81

True